# Walkthrough Challenge 3 - Chat with structured data

Duration: 30 minutes

## Overview
- In this challenge, you will start by configuring and initializing the kernel.
- [TODO]

## Prerequisites

- Please ensure that you have completed the [Setup](../setup/setup.ipynb) before starting this challenge.

### Task 1: Configure and Initialize Semantic Kernel

⚠️ Note: You should have already completed all tasks on the [Setup](../setup/setup.ipynb). If you have not, please go back and complete it now.

#### Step 1: Load Semantic Kernel settings

In this step, we will load the Semantic Kernel settings that we created in the [Setup](../setup/setup.ipynb) notebook.

In [8]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.0.1-preview"
#r "nuget: System.Data.SQLite, 1.0.118"
#r "nuget: Dapper, 2.1.24"

#!import ../setup/config/Settings.cs
#!import ../setup/config/Utils.cs
#!import plugins/SqlPlugin/SqlPlugin.cs

Installed Packages Dapper, 2.1.24 Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview Microsoft.SemanticKernel.Planners.OpenAI, 1.0.1-preview System.Data.SQLite, 1.0.118

#### Step 2: Initialize Semantic Kernel

In [9]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile("../setup/config/settings.json");

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

In [10]:
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.SemanticKernel.Planning;

#pragma warning disable SKEXP0061

var config = new FunctionCallingStepwisePlannerConfig
{
    MaxIterations = 15,
    MaxTokens = 8000,
};
var planner = new FunctionCallingStepwisePlanner();

In [11]:
kernel.ImportPluginFromObject(new SqlPlugin());

In [17]:
#pragma warning disable SKEXP0061

var ask = "What is the most sold product?";
var result = await planner.ExecuteAsync(kernel, ask);

Console.WriteLine("\nResult:\n");
Console.WriteLine(result.FinalAnswer);

Getting tables...
Querying database with query: SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;
Getting schema for OrderDetails...
Querying database with query: PRAGMA table_info(OrderDetails);
Getting schema for Products...
Querying database with query: PRAGMA table_info(Products);
Running query...
Querying database with query: SELECT P.ProductName, SUM(OD.Quantity) AS TotalQuantity
FROM OrderDetails AS OD
JOIN Products AS P ON OD.ProductID = P.ProductID
GROUP BY P.ProductName
ORDER BY TotalQuantity DESC
LIMIT 1
Result:

The most sold product is 'Gorgonzola Telino', with a total quantity of 458 units sold.
